# Importing libraries

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
from colorama import Fore, Style

from power.params import *
from power.ml_ops.data import get_data_with_cache, get_forecast_data, clean_forecast_data
from power.ml_ops.cross_val import get_folds, train_test_split, get_X_y_seq
from power.ml_ops.model import init_baseline_keras, compile_model, initialize_model, train_model
from power.utils import compress

# tensforflow
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.layers import Lambda

from sklearn.preprocessing import MinMaxScaler

# pd.set_option('display.max_rows', 500)

2024-04-15 09:48:56.178511: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-15 09:48:57.179164: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-15 09:48:57.184998: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-15 09:49:11.877473: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Process raw data

In [ ]:
raw_data = pd.read_csv('../raw_data/history_forecast_bulk_20171007_20240312.csv')

In [ ]:
def compress(df, **kwargs):
    """
    Reduces size of dataframe by downcasting numerical columns
    """
    input_size = df.memory_usage(index=True).sum()/ 1024
    print("new dataframe size: ", round(input_size,2), 'kB')

    in_size = df.memory_usage(index=True).sum()
    for type in ["float", "integer"]:
        l_cols = list(df.select_dtypes(include=type))
        for col in l_cols:
            df[col] = pd.to_numeric(df[col], downcast=type)
    out_size = df.memory_usage(index=True).sum()
    ratio = (1 - round(out_size / in_size, 2)) * 100

    print("optimized size by {} %".format(round(ratio,2)))
    print("new dataframe size: ", round(out_size / 1024,2), " kB")

    return df

In [ ]:
raw_data = compress(raw_data)

In [ ]:
df = raw_data.copy()
df.info()

In [ ]:
df = df[['forecast dt iso', 'slice dt iso', 'temperature', 'dew_point', 'pressure',
       'ground_pressure', 'humidity', 'clouds', 'wind_speed', 'wind_deg',
       'rain', 'snow', 'ice', 'fr_rain', 'convective', 'snow_depth',
       'accumulated', 'hours', 'rate', 'probability']]

In [ ]:
df['forecast dt iso'] = df['forecast dt iso'].str.replace('+0000 UTC', '')
df['slice dt iso'] = df['slice dt iso'].str.replace('+0000 UTC', '')


In [ ]:
df.tail()

In [ ]:
df = df[df['forecast dt iso'].str.contains('12:00:00')]
df

In [ ]:
df['forecast dt iso'] = pd.to_datetime(df['forecast dt iso'])
df['slice dt iso'] = pd.to_datetime(df['slice dt iso'])

In [ ]:
df.info()

In [ ]:
df_unique_dates = df['forecast dt iso'].unique()
df_unique_dates


In [ ]:
df[(df['forecast dt iso'] == df_unique_dates[0]) & (df['slice dt iso'].between(df_unique_dates[0] + timedelta(days=1) - timedelta(hours=12), df_unique_dates[0] + timedelta(days=1) + timedelta(hours=11)))]

In [ ]:
df_revised = []
for date in df_unique_dates:
    data = df[(df['forecast dt iso'] == date) & (df['slice dt iso'].between(date + timedelta(days=1) - timedelta(hours=12), date + timedelta(days=2) + timedelta(hours=11)))]
    df_revised.append(data)

In [ ]:
df_revised_ordered = pd.concat(df_revised, ignore_index=True)

In [ ]:
df_revised_ordered.info()

In [ ]:
pv_weather_df = df_revised_ordered[df_revised_ordered['slice dt iso'] <= '2022-12-31 23:00:00']

In [ ]:
pv_weather_df.info()

# Process data function

In [ ]:
def clean_forecast_data(forecast_df: pd.DataFrame) -> pd.DataFrame:
    """
    Initial has 3.3 M entries (everyday: 4 forecasts of 16 days ahead)
    Cleaning it to: - 1 forecast perday (at 12:00)
                    - 48 hours a day
                    - right now hardcoded to match last forecast day with
                     last day of PV data
    """
    df = compress(forecast_df)

    # get only 1 forecast per day and deal with uncommon UTC format
    df['forecast dt iso'] = df['forecast dt iso'].str.replace('+0000 UTC', '')
    df['slice dt iso'] = df['slice dt iso'].str.replace('+0000 UTC', '')

    df = df[df['forecast dt iso'].str.contains('12:00:00')]

    df['forecast dt iso'] = pd.to_datetime(df['forecast dt iso'])
    df['slice dt iso'] = pd.to_datetime(df['slice dt iso'])

    df_unique_dates = df['forecast dt iso'].unique()

    # reduce to 48h of weather forecast (from 00:00 to 23:00 each day)
    df_revised = []
    for date in df_unique_dates:
        data = df[(df['forecast dt iso'] == date) & (df['slice dt iso'].between(date + timedelta(days=1) - timedelta(hours=12), date + timedelta(days=2) + timedelta(hours=11)))]
        df_revised.append(data)

    df_revised_ordered = pd.concat(df_revised, ignore_index=True)

    # hard code the end date to match wiht PV data
    processed_df = df_revised_ordered[df_revised_ordered['slice dt iso'] <= '2022-12-31 23:00:00']

    return processed_df

In [ ]:
raw_data = get_forecast_data()

In [ ]:
pv_weather_df = clean_forecast_data(raw_data)

In [ ]:
pv_weather_df.info()

# Save processed data

In [ ]:
pv_weather_df.to_csv('../raw_data/weather_forecast_processed.csv')

# Load processed data

In [ ]:
pv_weather_df = pd.read_csv('../raw_data/weather_forecast_processed.csv')
pv_weather_df.info()

In [ ]:
data = pv_weather_df.copy()
data.rename(columns={'forecast dt iso':'utc_time', 'slice dt iso':'prediction_utc_time'}, inplace=True)
data['utc_time'] = pd.to_datetime(data['utc_time'])
data['prediction_utc_time'] = pd.to_datetime(data['prediction_utc_time'])
data

In [ ]:
input_date ='2020-12-06'
input_datetime = datetime.strptime(input_date, '%Y-%m-%d')
data[data.utc_time.dt.date == (input_datetime.date())].iloc[:,:10]

In [ ]:
input_date ='2020-06-30'
input_datetime = datetime.strptime(input_date, '%Y-%m-%d')

df_forecast_day_before_input_date = data[data.utc_time.dt.date == (input_datetime.date() - timedelta(days=1))].iloc[-24:,:]
df_forecast_input_date = data[data.utc_time.dt.date == input_datetime.date()].iloc[:24,:]
df_forecast = pd.concat([df_forecast_day_before_input_date, df_forecast_input_date], axis=0).reset_index(drop=True)

features = ['temperature', 'clouds', 'wind_deg', 'rain', 'snow',]
X=df_forecast[features]
fig, ax = plt.subplots(nrows=5, ncols=2, figsize= (16,9))
for idx, feature in enumerate(features):
    sns.boxplot(data=X, x=feature, ax=ax[idx,0], legend='auto')
    sns.histplot(data=X, x=feature, ax=ax[idx,1], bins =5)
# df_forecast.iloc[:,0:10]

In [ ]:
df_forecast.utc_time.nunique(), df_forecast.prediction_utc_time.nunique()


In [ ]:
df_forecast.utc_time.value_counts(), df_forecast.prediction_utc_time.value_counts()

In [ ]:
scaler= MinMaxScaler()
X[features] = scaler.fit_transform(X[features])
X

In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=2, figsize= (16,9))
for idx, feature in enumerate(features[-5:]):
    sns.boxplot(data=X, x=feature, ax=ax[idx,0], legend='auto')
    sns.histplot(data=X, x=feature, ax=ax[idx,1], bins =5)

# PSEUDO-CODE

when getting the 10_000 sequences, the dt index should be used to extract 48 observations and x features

rnow an ouput of shape (48,1)

should become an output (48, 1 + x)

call a data module.function to process the weather forecast features

concat the data somehow: np.concat? 


# test get weather feature function

In [50]:
def train(
        min_date = '2017-10-07 00:00:00',
        max_date = '2019-12-31 23:00:00',
        split_ratio: float = 0.02, # 0.02 represents ~ 1 month of validation data on a 2009-2015 train set
        learning_rate=0.02,
        batch_size = 32,
        patience = 5
    ) -> float:

    """
    - Download processed data from your BQ table (or from cache if it exists)
    - Train on the preprocessed dataset (which should be ordered by date)
    - Store training results and model weights

    Return val_mae as a float
    """

    print(Fore.MAGENTA + "\n⭐️ Use case: train" + Style.RESET_ALL)
    print(Fore.BLUE + "\nLoading preprocessed validation data..." + Style.RESET_ALL)


    # --First-- Load processed PV data using `get_data_with_cache` in chronological order
    query_pv = f"""
        SELECT *
        FROM {GCP_PROJECT}.{BQ_DATASET}.processed_pv
        ORDER BY utc_time
    """

    data_processed_pv_cache_path = Path(LOCAL_DATA_PATH).joinpath("processed", f"processed_pv.csv")
    data_processed_pv = get_data_with_cache(
        gcp_project=GCP_PROJECT,
        query=query_pv,
        cache_path=data_processed_pv_cache_path,
        data_has_header=True
    )

    # --Second-- Load processed Weather Forecast data in chronological order
    query_forecast = f"""
        SELECT *
        FROM {GCP_PROJECT}.{BQ_DATASET}.processed_weather_forecast
        ORDER BY forecast_dt_unixtime, slice_dt_unixtime
    """

    data_processed_forecast_cache_path = Path(LOCAL_DATA_PATH).joinpath("processed", f"processed_weather_forecast.csv")
    data_processed_forecast = get_data_with_cache(
        gcp_project=GCP_PROJECT,
        query=query_forecast,
        cache_path=data_processed_forecast_cache_path,
        data_has_header=True
    )


    # the processed PV data from bq needs to be converted to datetime object
    data_processed_pv.utc_time = pd.to_datetime(data_processed_pv.utc_time,utc=True)

    if data_processed_pv.shape[0] < 240:
        print("❌ Not enough processed data retrieved to train on")
        return None

    # Split the data into training and testing sets
    train_pv = data_processed_pv[(data_processed_pv['utc_time'] > min_date) \
                                 & (data_processed_pv['utc_time'] < max_date)]

    if data_processed_forecast.shape[0] < 240:
        print("❌ Not enough processed data retrieved to train on")
        return None

    # Split the data into training and testing sets
    train_forecast = data_processed_forecast

    X_train, y_train = get_X_y_seq(train_pv,
                                   train_forecast,
                                   number_of_sequences=100,
                                   input_length=48,
                                   output_length=24,
                                   gap_hours=12)

    return X_train, y_train

X_train, y_train = train()
X_train.shape, y_train.shape


⭐️ Use case: train

Loading preprocessed validation data...

Load data from local CSV...
✅ Data loaded, with shape (376944, 3)

Load data from local CSV...
✅ Data loaded, with shape (91704, 24)


((100, 48, 5), (100, 24, 1))

In [66]:
idx = np.random.randint(0, X_train.shape[0])
pd.DataFrame(X_train[idx]).iloc[-24:]

,0,1,2,3,4
24,0.000,15.230000,99.0,0.0,3.27
25,0.000,14.560000,88.0,0.0,2.81
26,0.000,13.810000,73.0,0.0,2.36
27,0.000,13.340000,57.0,0.0,2.07
28,0.007,13.440000,42.0,0.0,2.06
29,0.038,14.010000,32.0,0.0,2.26
30,0.106,14.890000,28.0,0.0,2.54
31,0.225,15.910000,32.0,0.0,2.78
32,0.307,17.030001,40.0,0.0,2.93
33,0.461,18.250000,48.0,0.0,2.97


In [ ]:
pv_weather_df = pd.read_csv('../raw_data/weather_forecast_processed.csv')

def compress(df, **kwargs):
    """
    Reduces size of dataframe by downcasting numerical columns
    """
    input_size = df.memory_usage(index=True).sum()/ 1024
    print("new dataframe size: ", round(input_size,2), 'kB')

    in_size = df.memory_usage(index=True).sum()
    for type in ["float", "integer"]:
        l_cols = list(df.select_dtypes(include=type))
        for col in l_cols:
            df[col] = pd.to_numeric(df[col], downcast=type)
    out_size = df.memory_usage(index=True).sum()
    ratio = (1 - round(out_size / in_size, 2)) * 100

    print("optimized size by {} %".format(round(ratio,2)))
    print("new dataframe size: ", round(out_size / 1024,2), " kB")

    return df

pv_weather_df = compress(pv_weather_df)

def get_weather_forecast_features(forecast: pd.DataFrame, input_date: str) -> pd.DataFrame:
    """
    returns the weather forecast data from historical weather forecast in Tempelhof
    input: - a processed forecast dataframe of shape (91704,21)
           - an input date (str: YYYY-MM-DD)
    output: a dataframe of shape (48, 21)
            -> first 24 rows: hourly (from 00:00 to 23:00) weather forecast
               of input_date +1 forecast on input_date -1 (at 12:00)
            -> second 24 rows: hourly (from 00:00 to 23:00) weather forecast
               of input_date +1 forecast on input_date (at 12:00)
    """
    forecast.rename(columns={'forecast_dt_iso':'utc_time',
                        'slice_dt_iso':'prediction_utc_time'},
                        inplace=True)
    forecast['utc_time'] = pd.to_datetime(forecast['utc_time'])
    forecast['prediction_utc_time'] = pd.to_datetime(forecast['prediction_utc_time'])

    input_datetime = datetime.strptime(input_date, '%Y-%m-%d')

    forecast_day_before_input_date = forecast[forecast.utc_time.dt.date == (input_datetime.date() - timedelta(days=1))].iloc[-24:,:]
    forecast_input_date = forecast[forecast.utc_time.dt.date == input_datetime.date()].iloc[:24,:]
    df_forecast = pd.concat([forecast_day_before_input_date,
                             forecast_input_date], axis=0).reset_index(drop=True)
    return df_forecast

test_df = get_weather_forecast_features(pv_weather_df, '2020-06-30')
test_df.iloc[:,:5]

# try editing sequence 

In [ ]:
def get_Xi_yi(
    pv_fold:pd.DataFrame,
    forecast_fold:pd.DataFrame,
    input_length:int,       # 48
    output_length:int,      # 24
    gap_hours):
    '''
    - given a fold, it returns one sequence (X_i, y_i)
    - with the starting point of the sequence being chosen at random
    - TARGET is the variable(s) we want to predict (name of the column(s))
    '''
    TARGET = 'electricity'
    first_possible_start = 0
    last_possible_start = len(pv_fold) - (input_length + gap_hours + output_length) + 1

    random_start = np.random.randint(first_possible_start, last_possible_start)

    # input_start & input_end are the indexes of the 48h (training length) of training data
    # thus for weather forecast data from input_end should be used to add
    # the weather forecast features
    input_start = random_start
    input_end = random_start + input_length
    # here we extract the forecast date and hour
    forecast_date = pv_fold.iloc[input_end]['utc_time'].strftime('%Y-%m-%d')
    forecast_hour = pv_fold.iloc[input_end]['utc_time'].hour

    target_start = input_end + gap_hours
    target_end = target_start + output_length

    # first we parse the electricity feature
    # need to reset index only for X_i in order to be able to concat with X_weather later on
    X_i = pv_fold.iloc[input_start:input_end].reset_index()
    y_i = pv_fold.iloc[target_start:target_end][[TARGET]]    # creates a pd.DataFrame for the target y

    # then we parse/create the weather forecast features
    #TODO : proper data
    X_weather = get_weather_forecast_features(forecast_fold, forecast_date)

    #
    features = ['temperature', 'clouds', 'wind_speed']
    to_concat = [X_i['electricity'], X_weather[features]]
    X_i = pd.concat(to_concat, axis=1)
    return (X_i, y_i)

min_date = '2017-10-07 12:00:00'
max_date = '2019-12-31 23:00:00'
query = f"""
        SELECT *
        FROM {GCP_PROJECT}.{BQ_DATASET}.processed_pv
        ORDER BY utc_time
    """

data_processed_cache_path = Path(LOCAL_DATA_PATH).joinpath("processed", f"processed_pv.csv")
data_processed = get_data_with_cache(
    gcp_project=GCP_PROJECT,
    query=query,
    cache_path=data_processed_cache_path,
    data_has_header=True
)

# the processed data from bq needs to be converted to datetime object
data_processed.utc_time = pd.to_datetime(data_processed.utc_time,utc=True)

if data_processed.shape[0] < 240:
    print("❌ Not enough processed data retrieved to train on")
    # return None

# Split the data into training and testing sets
train = data_processed[(data_processed['utc_time'] > min_date) \
                        & (data_processed['utc_time'] < max_date)]
# train = data_processed[data_processed['utc_time'] < max_date]

# train = train[['electricity']]

fold = train.copy()
number_of_sequences=100
input_length=48
output_length=24
gap_hours=12

X, y = [], []

for i in range(number_of_sequences):
    (Xi, yi) = get_Xi_yi(fold, input_length, output_length, gap_hours)
    X.append(Xi)
    y.append(yi)

X_train, y_train = np.array(X), np.array(y)

X_train.shape

In [ ]:
X_train.shape[1:], y_train.shape[1]


# For Jerome - ignore the rest

In [ ]:
pv_weather_df = pv_weather_df.rename(columns={'forecast dt iso': 'date_of_forcast',
                                              'slice dt iso': 'forcasting_date_range',
                                              'fr_rain': 'freezing_rain_vol',
                                            })

In [ ]:
pv_df

In [ ]:
comb_df = pd.merge(pv_weather_df, pv_df, )

In [ ]:
pv_df = df.copy()

In [ ]:
min_date = '1980-01-01 00:00:00'
max_date = '2019-12-31 23:00:00'
train = pv_df[pv_df['utc_time'] <= max_date]
test = pv_df[pv_df['utc_time'] > max_date]

In [ ]:
train = train[['electricity']]
test = test[['electricity']]

# Cross validation

In [ ]:
TARGET = 'electricity'
fold_length = 43800             # 5 years
fold_stride = 43800             # 5 years
train_test_ratio = 0.8          # 5 yrs/6 yrs
input_length = 48               # number of obsevations per one sequence
output_length = 24              # Day-ahead predictions
n_seq_train = 500               # number_of_sequences_train
n_seq_val = 100                 # number_of_sequences_test
n_unit = 48                     # number of hidden units
learning_rate = 0.02
patience = 5
epochs = 50
batch_size = 32

In [ ]:
def cross_validate_baseline_and_lstm():

    list_of_mae_baseline_model = []
    list_of_mae_recurrent_model = []

    # 0 - Creating folds
    # =========================================
    folds = get_folds(train, fold_length, fold_stride)

    for fold_id, fold in enumerate(folds):

        # 1 - Train/Test split the current fold
        # =========================================
        (fold_train, fold_val) = train_test_split(fold, train_test_ratio, input_length)

        X_train, y_train = get_X_y_seq(fold_train, n_seq_train, input_length, output_length, gap_hours=12)
        X_val, y_val = get_X_y_seq(fold_val, n_seq_val, input_length, output_length, gap_hours=12)

        # 2 - Modelling
        # =========================================

        ##### Baseline Model
        baseline_model = init_baseline_keras()
        mae_baseline = baseline_model.evaluate(X_val, y_val, verbose=0)[1]
        list_of_mae_baseline_model.append(mae_baseline)
        print("-"*50)
        print(f"MAE baseline fold n°{fold_id} = {round(mae_baseline, 2)}")

        ##### LSTM Model
        model = initialize_model(X_train, y_train, n_unit=n_unit)
        model = compile_model(model, learning_rate=learning_rate)
        model, history = train_model(model,
                                     X_train,
                                     y_train,
                                     validation_split = 0.2,
                                     batch_size = batch_size,
                                     epochs = epochs)

        # Create a figure and axes object with 1 row and 2 columns
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

        # Plot training & validation MAE values
        axes[0].plot(history.history['mae'])
        axes[0].plot(history.history['val_mae'])
        axes[0].set_title('Model MAE')
        axes[0].set_ylabel('MAE')
        axes[0].set_xlabel('Epoch')
        axes[0].legend(['Train', 'Validation'], loc='upper left')

        # Plot training & validation loss values
        axes[1].plot(history.history['loss'])
        axes[1].plot(history.history['val_loss'])
        axes[1].set_title('Model Loss')
        axes[1].set_ylabel('Loss')
        axes[1].set_xlabel('Epoch')
        axes[1].legend(['Train', 'Validation'], loc='upper left')

        # Adjust layout to prevent overlap
        plt.tight_layout()

        # Show the plot
        plt.show()

        res = model.evaluate(X_val, y_val, verbose=0)    # evaluating LSTM (metric)
        mae_lstm = res[1]
        list_of_mae_recurrent_model.append(mae_lstm)
        print(f"MAE LSTM fold n°{fold_id} = {round(mae_lstm, 2)}")

        ##### Comparison LSTM vs Baseline for the current fold
        print(f"improvement over baseline: {round((1 - (mae_lstm/mae_baseline))*100,2)} % \n")

    return list_of_mae_baseline_model, list_of_mae_recurrent_model


In [ ]:
mae_baselines, mae_lstms = cross_validate_baseline_and_lstm()

In [ ]:
print(f"average percentage improvement over baseline = {round(np.mean(1 - (np.array(mae_lstms)/np.array(mae_baselines))),2)*100}%")

# Prediction

In [ ]:
X_train, y_train = get_X_y_seq(train, number_of_sequences=10000, input_length=48, output_length=24, gap_hours=12)

In [ ]:
model = initialize_model(X_train, y_train, n_unit=n_unit)
model = compile_model(model, learning_rate=learning_rate)
model, history = train_model(model,
                                X_train,
                                y_train,
                                validation_split = 0.1,
                                batch_size = batch_size,
                                epochs = epochs)

In [ ]:
from power.params import *
from power.ml_ops.data import get_data_with_cache
from pathlib import Path
import tensorflow as tf

input_pred = '2021-05-08 12:00:00'

query = f"""
    SELECT *
    FROM {GCP_PROJECT}.{BQ_DATASET}.processed_pv
    ORDER BY utc_time
"""

data_processed_cache_path = Path(LOCAL_DATA_PATH).joinpath("processed", f"processed_pv.csv")
data_processed = get_data_with_cache(
    gcp_project=GCP_PROJECT,
    query=query,
    cache_path=data_processed_cache_path,
    data_has_header=True
)

X_pred = data_processed[data_processed['utc_time'] < input_pred][-48:]

In [ ]:
X_pred = X_pred[['electricity']]

In [ ]:
X_pred = X_pred.to_numpy()
X_pred_tf = tf.convert_to_tensor(X_pred)
X_pred_tf = tf.expand_dims(X_pred_tf, axis=0)

In [ ]:
y_pred = model.predict(X_pred_tf)

In [ ]:
y_pred[0]

In [ ]:
plt.plot(y_pred[0])